# Data Analysis and Feature Engineering

In this notebook you can find the analysis of our `train.csv` dataset. We will utilize this knowledge to do some feature engineering. We will maybe do some tests on the datasets we create to see if the features are actually functional or not.

But first things first, lets load the dataset and get a basic understanding.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.shape

(188533, 13)

In [4]:
data.describe()

,id,model_year,milage,price
count,188533.000000,188533.000000,188533.000000,1.885330e+05
mean,94266.000000,2015.829998,65705.295174,4.387802e+04
std,54424.933488,5.660967,49798.158076,7.881952e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,47133.000000,2013.000000,24115.000000,1.700000e+04
50%,94266.000000,2017.000000,57785.000000,3.082500e+04
75%,141399.000000,2020.000000,95400.000000,4.990000e+04
max,188532.000000,2024.000000,405000.000000,2.954083e+06


In [5]:
data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [6]:
for col in data.columns:
    print(col, np.sum(data[col].isna()))

id 0
brand 0
model 0
model_year 0
milage 0
fuel_type 5083
engine 0
transmission 0
ext_col 0
int_col 0
accident 2452
clean_title 21419
price 0


This is something we will have to look into. Since we have a lot of data we can think about just dropping the rows with missing values, as they are only a fraction of the dataset. 

For fuel type we could just fill the missing values up with the most common fuel. Alternatively one could write a script that takes the brand and the model of the car and automaticly searches for the correct fueltype. However, this might be a bit out of the scope for this project.

For accidents it would make sense to just fill the missing values with 'None reported', as this would make sense from the semantics.

I am not sure what clean title means. This might be in the sense that it has passed some kind of enviromental test, but we will have to look further into this.

In [7]:
for col in data.columns:
    if not pd.api.types.is_numeric_dtype(data[col]):
        print(col, len(data[col].unique()))

brand 57
model 1897
fuel_type 8
engine 1117
transmission 52
ext_col 319
int_col 156
accident 3
clean_title 2


In [8]:
for col in data.columns:
    if not pd.api.types.is_numeric_dtype(data[col]):
        if len(data[col].unique()) < 9:
            print(col, data[col].unique())

fuel_type ['Gasoline' 'E85 Flex Fuel' nan 'Hybrid' 'Diesel' 'Plug-In Hybrid' '–'
 'not supported']
accident ['None reported' 'At least 1 accident or damage reported' nan]
clean_title ['Yes' nan]


From the looks of it we can say a few things. The missing fuel types might be 'electrical', because this isn't a class and maybe this wasn't considered to be a fuel. We will validate or disprove this theory later by looking at some examples where the fuel type is missing and manually looking up the fuel type for the model. Another option might be to cross reference in the table if there is another instance of this model with the fuel type given and then jsut use this.

Moreover we will take 'nan' in the clean title as a no, because otherwise this column wouldn't contain any information because all the given values are 'Yes'.

For the accidents we could either consider assigning the 'nan' values the value 'None reported' or just fill them up randomly with the distribution of 'None reported' and the other value.

## Engine

We think there can be a lot of feature engineering done with the engine category. Firstly one can split of the horsepower into a seperate column.

In [9]:
data_engine = pd.DataFrame(data[['id', 'engine']].copy())

In [10]:
regex = 'HP'
data_engine['horsepower'] = np.nan
for ind, engine in enumerate(data_engine['engine']):
    if regex in engine:
        val = engine.split(regex)
        data_engine.loc[ind, 'horsepower'] = np.float64(val[0])
        data_engine.loc[ind, 'engine'] = val[1]
print(np.sum(data_engine['horsepower'].isna()))
data_engine.head()

33259


,id,engine,horsepower
0,0,1.6L 4 Cylinder Engine Gasoline Fuel,172.0
1,1,3.9L 8 Cylinder Engine Gasoline Fuel,252.0
2,2,5.3L 8 Cylinder Engine Flex Fuel Capability,320.0
3,3,5.0L 8 Cylinder Engine Gasoline Fuel,420.0
4,4,2.0L 4 Cylinder Engine Gasoline Fuel,208.0


We will have to think about what we should do with the missing values. One possibility may be to again use cross references in the table to find another model, however it may be, that the same model has a different engine and can therefore not be simply used. Another option would be to use the average horsepower or maybe the average horsepower of some subset of the dataset (e.g. all instances of this model/manufacturer).

Next we will split of the liters that are being used by the engine.

In [11]:
regexes = ['L ', 'Liter']
data_engine['liters'] = np.nan
for regex in regexes:
    for ind, engine in enumerate(data_engine['engine']):
        if regex in engine:
            val = engine.split(regex)
            data_engine.loc[ind, 'liters'] = np.float64(val[0])
            data_engine.loc[ind, 'engine'] = val[1]

print(np.sum(data_engine['liters'].isna()))
data_engine.head()

6845


,id,engine,horsepower,liters
0,0,4 Cylinder Engine Gasoline Fuel,172.0,1.6
1,1,8 Cylinder Engine Gasoline Fuel,252.0,3.9
2,2,8 Cylinder Engine Flex Fuel Capability,320.0,5.3
3,3,8 Cylinder Engine Gasoline Fuel,420.0,5.0
4,4,4 Cylinder Engine Gasoline Fuel,208.0,2.0


Now for the final we will filter out the cylinder count.

In [12]:
import copy

data_engine_test = data_engine.copy()

regex = 'Cylinder'
data_engine_test['cylinder'] = np.nan
data_engine_test['V'] = 0
data_engine_test['I'] = 0
data_engine_test['dual'] = 0

data_engine_test.head()

,id,engine,horsepower,liters,cylinder,V,I,dual
0,0,4 Cylinder Engine Gasoline Fuel,172.0,1.6,NaN,0,0,0
1,1,8 Cylinder Engine Gasoline Fuel,252.0,3.9,NaN,0,0,0
2,2,8 Cylinder Engine Flex Fuel Capability,320.0,5.3,NaN,0,0,0
3,3,8 Cylinder Engine Gasoline Fuel,420.0,5.0,NaN,0,0,0
4,4,4 Cylinder Engine Gasoline Fuel,208.0,2.0,NaN,0,0,0


In [13]:
# Don't worry if this cell takes some time to execute

def get_cylinder_count(string):
    while not string[0].isdigit():
        string = string[1:]
    return np.int32(string)

def check_for_special_engine(string):
    orig_engine = copy.copy(string)
    type_engine = None
    if type(string) != str or len(string) == 0:
        return None, np.nan, np.nan
    if string[0] == 'I' or 'Straight' in string:
        type_engine = 'I'
    elif string[0] == 'V':
        type_engine = 'V'
    elif 'Electric' in engine:
        return None, np.nan, np.nan
    elif 'Dual' in engine:
        return 'dual', np.nan, np.nan
    
    number = str()
    while not string[0].isdigit():
        string = string[1:]
        if len(string) == 0:
            return type_engine, np.nan, orig_engine
    while string[0].isdigit():
        number += string[0]
        string = string[1:]
        if len(string) == 0:
            return type_engine, np.int32(number), np.nan
    if 'Cylinder' in string:
        string = string.split('Cylinder')[1]
    return type_engine, np.int32(number), string

for ind, engine in enumerate(data_engine_test['engine']):
    type_engine, cylinders, engine = check_for_special_engine(engine)
    data_engine_test.loc[ind, 'engine'] = engine
    data_engine_test.loc[ind, 'cylinder'] = cylinders
    if type_engine != None:
        data_engine_test.loc[ind, type_engine] = 1
    else:
        # later we can add the this would make fuel electric
        pass

print(np.sum(data_engine_test['cylinder'].isna()))
data_engine_test.head()

15962


,id,engine,horsepower,liters,cylinder,V,I,dual
0,0,Engine Gasoline Fuel,172.0,1.6,4.0,0,0,0
1,1,Engine Gasoline Fuel,252.0,3.9,8.0,0,0,0
2,2,Engine Flex Fuel Capability,320.0,5.3,8.0,0,0,0
3,3,Engine Gasoline Fuel,420.0,5.0,8.0,0,0,0
4,4,Engine Gasoline Fuel,208.0,2.0,4.0,0,0,0


We will now take a look at what the remaining parts of the engine contain as information. For example `Engine Gasoline Fuel` doesn't contain any more information, as this is already given by the fuel type. But `32V PDI DOHC` has still some interesting information, as these are some special features for the engine. We will now try to see what other information is contained in the string.

In [14]:
engine_infos = set()

for engine in data_engine_test['engine']:
    if type(engine) == str:
        infos = engine.split(' ')
        
        for info in infos:
            engine_infos.add(info)
    elif not pd.isna(engine):
        print(engine)

print(engine_infos)

{'', 'Gasoline/Mild', 'Electric', 'L/406', '18V', 'L/122', '/', '160kW', '64V', '(697V)', 'Battery', 'Electric/Gas', 'GTDI', '.5L', 'Gas/Electric', 'Flexible', 'Range', 'AH', 'FR', 'T/C', 'TSI', '70kW', 'Fuel', 'SOHC', 'OHV', '48V', 'DOHC', 'Turbo', 'Standard', 'V', '12V', '2.0', '.2Ah', 'Twin', 'SC', 'Flex', 'Gasoline', 'Engine', 'GDI', '24V', 'DDI', 'Rotary', '.0L', 'ULEV', 'Plug-In', 'RR', '40V', 'engine', 'I-VTEC', '6.7', 'PDI', 'Hybrid', 'TFSI', '16V', 'MPFI', 'Supercharged', '–', 'V6', '32V', 'Diesel', 'SIDI', 'Capability'}


In [15]:
data_engine_test.describe()

,id,horsepower,liters,cylinder,V,I,dual
count,188533.000000,155274.000000,181688.000000,172571.000000,188533.000000,188533.000000,188533.000000
mean,94266.000000,343.263856,3.775477,6.378679,0.346852,0.097007,0.000621
std,54424.933488,114.734210,1.352121,1.835269,0.475969,0.295968,0.024904
min,0.000000,70.000000,0.650000,2.000000,0.000000,0.000000,0.000000
25%,47133.000000,263.000000,3.000000,6.000000,0.000000,0.000000,0.000000
50%,94266.000000,329.000000,3.500000,6.000000,0.000000,0.000000,0.000000
75%,141399.000000,416.000000,4.700000,8.000000,1.000000,0.000000,0.000000
max,188532.000000,1020.000000,8.400000,120.000000,1.000000,1.000000,1.000000


Here we can see some interesting things. Firstly there are a lot of different upgrades we can use as features. Secondly there are also values like `.5L` or `V6` that might indicate, that we have made some mistakes in the previous steps. But we don't think that these are exceptions and therefore focus on them later on. We will now remove values that don't contain any further information as discussed earlier.

We won't use all of these features with one hot encoding. We will turn the `xV` values like `32V` into one feature, since these can be considered as a numeric value.

Another interesting thing is that we can calculate the `horsepower` from the kW value given in the engine.

In [16]:
engine_infos.discard('Fuel')
engine_infos.discard('Electric')
engine_infos.discard('Diesel')
engine_infos.discard('Gasoline')
engine_infos.discard('Hybrid')
engine_infos.discard('Engine')
engine_infos.discard('engine')
engine_infos.discard('Standard')
engine_infos.discard('_')
engine_infos.discard('.5L')
engine_infos.discard('.0L')

print(engine_infos)

{'', 'Gasoline/Mild', 'L/406', '18V', 'L/122', '/', '160kW', '64V', '(697V)', 'Battery', 'Electric/Gas', 'GTDI', 'Gas/Electric', 'Flexible', 'Range', 'AH', 'FR', 'T/C', 'TSI', '70kW', 'SOHC', 'OHV', '48V', 'DOHC', 'Turbo', 'V', '12V', '2.0', '.2Ah', 'Twin', 'SC', 'Flex', 'GDI', '24V', 'DDI', 'Rotary', 'ULEV', 'Plug-In', 'RR', '40V', 'I-VTEC', '6.7', 'PDI', 'TFSI', '16V', 'MPFI', 'Supercharged', '–', 'V6', '32V', 'SIDI', 'Capability'}


In [17]:
for ind, engine in enumerate(data_engine_test['engine']):
    if not pd.isna(engine):
        if '6.7' in engine:
            print(data.loc[ind])

id                                                 727
brand                                             Ford
model                                     F-250 LARIAT
model_year                                        2017
milage                                           44261
fuel_type                                       Diesel
engine          Intercooled Turbo Diesel V-8 6.7 L/406
transmission                                 Automatic
ext_col                                          White
int_col                                          Black
accident                                 None reported
clean_title                                        NaN
price                                            51598
Name: 727, dtype: object
id                                                8914
brand                                             Ford
model                                     F-250 LARIAT
model_year                                        2022
milage                                  

When we look at this output we see that our previous processing steps had some issues. We might change the order of our data processing steps. We will now look at the special configuration with the original data.

In [18]:
data_engine = data[['id', 'engine']].copy()

engine_infos = set()

for engine in data_engine['engine']:
    if type(engine) == str:
        infos = engine.split(' ')
        
        for info in infos:
            engine_infos.add(info)
    elif not pd.isna(engine):
        print(engine)

print(engine_infos)

{'435.0HP', '142.0HP', '-', '246.0HP', '422.0HP', '293.0HP', '192.0HP', '217.0HP', 'I3', '582.0HP', '12V', '186.0HP', '5.9L', 'I4', 'SC', '126.0HP', 'Flex', '104.0HP', 'I-VTEC', '645.0HP', '320.0HP', '3.0L', '332.0HP', '166.0HP', '16V', '362.0HP', '553.0HP', '660.0HP', '148.0HP', 'Diesel', '545.0HP', '403.0HP', '6.6', '429.0HP', '211.0HP', '185.0HP', '6.7L', '205.0HP', '198.0HP', '342.0HP', '4.7L', '158.0HP', '155.0HP', '201.0HP', '385.0HP', '4.6', '353.0HP', '214.0HP', '540.0HP', '220.0HP', '70kW', '273.0HP', '150.0HP', '225.0HP', '5.4L', '260.0HP', 'Turbo', '562.0HP', '253.0HP', '502.0HP', '266.0HP', '303.0HP', '70.0HP', 'Rotary', '308.0HP', '197.0HP', '389.0HP', 'engine', '2.2L', '247.0HP', '283.0HP', '256.0HP', '237.0HP', '550.0HP', '281.0HP', '178.0HP', 'Capability', '1.4L', '1.8', '7.0L', '344.0HP', '165.0HP', 'Electric/Gas', '12', '670.0HP', '282.0HP', '264.0HP', '411.0HP', '5.7L', '415.0HP', '426.0HP', '120', '454.0HP', 'Hydrogen', '172.0HP', '271.0HP', '393.0HP', '610.0HP', '2

In the new approach we can firstly filter out the horsepower values and the liter infos for the engines.

In [19]:
engine_infos = set()
data_engine['horsepower'] = 0
data_engine['liters'] = np.nan
data_engine['cylinders'] = np.nan
data_engine['valves'] = np.nan
data_engine['V'] = 0
data_engine['I'] = 0
data_engine['H'] = 0
data_engine['W'] = 0

unneccessary_infos = ['Fuel',
                      'Electric',
                      'Diesel',
                      'Gasoline',
                      'Hydrogen',
                      'Cylinder',
                      'Hybrid',
                      'PlugIn',
                      'Engine',
                      'engine',
                      'Standard',
                      'Motor',
                      'Motors',
                      'Capability',
                      'System',
                      'Liter', 
                      'Gasoline/Mild', # This might contain information for hybrid cars, we will maybe look into it later on
                      'Gas/Electric', # This might contain information for hybrid cars, we will maybe look into it later on
                      'Electric/Gas', # This might contain information for hybrid cars, we will maybe look into it later on
                      '-',
                      '/',
                      '–'
                     ]

for ind, engine in enumerate(data_engine['engine']):
    if type(engine) == str:
        infos = engine.split(' ')
        if infos[-1] == 0:
            print('DEBUG')
            infos = infos[:-1]
        used_infos = []
        for info in infos:
            """
            if info == 0 and infos[-1] == 0:
                print(infos[-1])
                print(data.loc[ind, 'engine'])
                print(data_engine.loc[ind, 'engine'])
                print(infos)
            """
            info = str(info)
            if '-' in info:
                used_infos.append(0)
                used_infos[-1] = info
                infos.append(0)
                info = str(info.replace('-', ''))
                infos[1] = info
            
            if info in unneccessary_infos or len(info) == 0:
                used_infos.append(0)
                used_infos[-1] = info
                
            elif info[-2:] == 'HP':
                data_engine.loc[ind, 'horsepower'] = np.float32(info[:-2])
                used_infos.append(0)
                used_infos[-1] = info
                
            elif info.isnumeric(): # This one checks for the cylinder count
                #print(info)
                data_engine.loc[ind, 'cylinders'] = np.int32(info)
                used_infos.append(0)
                used_infos[-1] = info
            
            elif not info[0].isnumeric() and info[1:].isnumeric(): # This one checks for the cylinder count
                data_engine.loc[ind, 'cylinders'] = np.int32(info[1:])
                data_engine.loc[ind, info[0]] = 1
                used_infos.append(0)
                used_infos[-1] = info

            elif info == 'Straight':
                data_engine.loc[ind, 'I'] = 1
                used_infos.append(0)
                used_infos[-1] = info
                
            elif info[-1] == 'L':
                #print(info)
                data_engine.loc[ind, 'liters'] = np.float32(info[:-1])
                used_infos.append(0)
                used_infos[-1] = info
                
            elif info.replace('.','').isnumeric(): # This one checks for the liters
                data_engine.loc[ind, 'liters'] = np.float32(info)
                used_infos.append(0)
                used_infos[-1] = info
            elif info[-1] == 'V' and info[:-1].isnumeric():
                data_engine.loc[ind, 'valves'] = np.int32(info[:-1])
                used_infos.append(0)
                used_infos[-1] = info
            elif info[-2:] == 'kW':
                data_engine.loc[ind, 'horsepower'] += np.float32(info[:-2]) * 1.34102 # Formula to get hp from kW
                used_infos.append(0)
                used_infos[-1] = info
            else:
                if info == 'Flexible':
                    info = 'Flex'
                    used_infos.append(0)
                    used_infos[-1] = 'Flexible'
                    infos.append(0)
                    infos[-1] = info

        remaining_engine = [str(eng) for eng in infos if eng not in used_infos and str(eng) != '0']
        for eng in remaining_engine:
            engine_infos.add(eng)

        #print(remaining_engine)
        data_engine.loc[ind, 'engine'] = ' '.join(remaining_engine)

    elif not pd.isna(engine):
        print(engine)

print(engine_infos)
data_engine.head()

/tmp/ipykernel_11707/723478252.py:99: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '93.87139892578125' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_engine.loc[ind, 'horsepower'] += np.float32(info[:-2]) * 1.34102 # Formula to get hp from kW


{'L/406', 'L/122', '(697V)', 'Battery', 'Intercooled', 'GTDI', 'AC', 'Flat', 'Dual', 'Range', 'AH', 'FR', 'T/C', 'TSI', 'SOHC', 'OHV', 'DOHC', 'Turbo', '111.2Ah', 'Premium', 'Unleaded', 'Twin', 'SC', 'Flex', 'IVTEC', 'GDI', 'DDI', 'Rotary', 'ULEV', 'RR', 'PDI', 'TFSI', 'MPFI', 'Supercharged', 'SIDI'}


,id,engine,horsepower,liters,cylinders,valves,V,I,H,W
0,0,,172.0,1.6,4.0,NaN,0,0,0,0
1,1,,252.0,3.9,8.0,NaN,0,0,0,0
2,2,Flex,320.0,5.3,8.0,NaN,0,0,0,0
3,3,,420.0,5.0,8.0,NaN,0,0,0,0
4,4,,208.0,2.0,4.0,NaN,0,0,0,0


In [20]:
for info in engine_infos:
    data_engine[info] = 0

for ind, engine in enumerate(data_engine['engine']):
    if len(engine) > 0:
        infos = engine.split(' ')
        for info in infos:
            data_engine.loc[ind, info] = 1
data_engine.head()

,id,engine,horsepower,liters,cylinders,valves,V,I,H,W,...,GDI,DDI,Rotary,ULEV,RR,PDI,TFSI,MPFI,Supercharged,SIDI
0,0,,172.0,1.6,4.0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,,252.0,3.9,8.0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Flex,320.0,5.3,8.0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,,420.0,5.0,8.0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,,208.0,2.0,4.0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


This approach seems to be working way better, so will use this approach to extracting the information fromthe engine feature.

As a final step we will have to fill up the missing values here. For the cylinders we will just use the mode of the column. For the valves we will use the formula: $valves = cylinders * 4$ as this is often the cast. For the liters and the horsepower we will take the average of the engines with the same amount of cylinders.

In [21]:
# This cell takes quite a long time to run

data_engine['cylinders'] = [val if not pd.isna(val) else data_engine['cylinders'].mode()[0] for val in data_engine['cylinders']]

data_engine['valves'] = [val if not pd.isna(val) else 4* data_engine.loc[ind, 'cylinders'] for ind, val in enumerate(data_engine['valves'])]

for ind, val in enumerate(data_engine['liters']):
    if pd.isna(val):
        data_engine.loc[ind, 'liters'] = data_engine[data_engine['cylinders'] == data_engine.loc[ind, 'cylinders']]['liters'].mean()

for ind, val in enumerate(data_engine['horsepower']):
    if val == 0:
        data_engine.loc[ind, 'horsepower'] = data_engine[data_engine['cylinders'] == data_engine.loc[ind, 'cylinders']]['horsepower'].mean() 

data_engine.drop(['id', 'engine'], axis=1, inplace=True)
data_engine.head()

,horsepower,liters,cylinders,valves,V,I,H,W,L/406,L/122,...,GDI,DDI,Rotary,ULEV,RR,PDI,TFSI,MPFI,Supercharged,SIDI
0,172.0,1.6,4.0,16.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,252.0,3.9,8.0,32.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,320.0,5.3,8.0,32.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,420.0,5.0,8.0,32.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,208.0,2.0,4.0,16.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
data_engine.head(50)

,horsepower,liters,cylinders,valves,V,I,H,W,L/406,L/122,...,GDI,DDI,Rotary,ULEV,RR,PDI,TFSI,MPFI,Supercharged,SIDI
0,172.000000,1.600000,4.0,16.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,252.000000,3.900000,8.0,32.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,320.000000,5.300000,8.0,32.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,420.000000,5.000000,8.0,32.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,208.000000,2.000000,4.0,16.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,252.000000,2.000000,4.0,16.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,333.000000,3.000000,6.0,24.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,355.000000,5.300000,8.0,32.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,265.229646,2.700000,6.0,24.0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,425.000000,3.000000,6.0,24.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Other features

Lets now take a look at the other features.

In [47]:
data_others = data.copy()

data_others.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


Lets firstly encode the `clean_title` numericaly.

In [48]:
data_others['clean_title'] = [1 if val == 'Yes' else 0 for val in data_others['clean_title']]

In [49]:
data_others['accident'] = [1 if val == 'At least 1 accident or damage reported' else 0 for val in data_others['accident']]
data_others.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,0,1,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,1,1,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,0,1,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,0,1,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,0,1,97500


Lets now take a look at the missing `fuel_type` values.

In [50]:
data_missing_fuel = data_others[data_others['fuel_type'].isna()]
data_missing_fuel.head(10)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
11,11,Tesla,Model S P100D,2015,81500,NaN,Electric Motor Electric Fuel System,1-Speed A/T,Gray,White,0,1,19000
32,32,Tesla,Model S P100D,2018,30300,NaN,518.0HP Electric Motor Electric Fuel System,A/T,White,White,0,1,64000
51,51,Tesla,Model 3 Long Range,2020,38360,NaN,Dual Motor - Standard,Automatic,White,Black,0,0,59598
52,52,Tesla,Model X 75D,2021,88000,NaN,518.0HP Electric Motor Electric Fuel System,1-Speed A/T,Black,Black,0,1,53600
161,161,Tesla,Model S Long Range,2021,9500,NaN,670.0HP Electric Motor Electric Fuel System,A/T,White,Beige,0,1,89999
174,174,Porsche,Taycan Turbo,2023,1794,NaN,Electric Motor Electric Fuel System,2-Speed A/T,Blue,Black,0,1,83500
176,176,Nissan,Leaf S,2020,14330,NaN,Electric,1-Speed Automatic,Brilliant Silver Metallic,Black,0,0,52999
240,240,Tesla,Model Y Performance,2020,46897,NaN,455.0HP Electric Motor Electric Fuel System,1-Speed A/T,Gray,White,0,1,29000
283,283,Tesla,Model Y Performance,2023,7578,NaN,Electric,Automatic,White,Black,0,0,267950
379,379,Tesla,Model X 75D,2023,5338,NaN,518.0HP Electric Motor Electric Fuel System,A/T,Gray,White,0,1,97500


Here we can see that a lot of the missing data points come from electric cars. If there is a `Electric` in the engine it seems we can assume that the car is an electric vehical. So lets look at the points where the engine doesn't contain this information.

In [51]:
data_non_electric = data_missing_fuel[-data_missing_fuel['engine'].str.contains('Electric', na=False)]
data_non_electric.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
51,51,Tesla,Model 3 Long Range,2020,38360,NaN,Dual Motor - Standard,Automatic,White,Black,0,0,59598
3286,3286,smart,ForTwo Pure,2008,156875,NaN,70.0HP 1.0L 3 Cylinder Engine Gasoline Fuel,2-Speed A/T,Black,Gray,1,1,9000
6794,6794,Chevrolet,Corvette Stingray w/2LT,2023,10900,NaN,490.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,Blue,Brown,0,1,67900
7233,7233,Tesla,Model Y Long Range,2021,35315,NaN,Dual Motor - Standard,Automatic,Silver,Black,0,0,47598
7362,7362,Tesla,Model S 85D,2015,75897,NaN,Dual Motor - Standard,Automatic,Silver,Black,0,0,18998


Here we can see that some of the datapoints come from the brand `Tesla` who only build electrical cars. Therefore lets remove these as well.

In [52]:
print(data_non_electric[data_non_electric['brand'] != 'Tesla'].shape)
data_non_electric[data_non_electric['brand'] != 'Tesla'].head()

(124, 13)


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
3286,3286,smart,ForTwo Pure,2008,156875,NaN,70.0HP 1.0L 3 Cylinder Engine Gasoline Fuel,2-Speed A/T,Black,Gray,1,1,9000
6794,6794,Chevrolet,Corvette Stingray w/2LT,2023,10900,NaN,490.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,Blue,Brown,0,1,67900
8879,8879,Audi,Q5 S line Premium Plus,2016,35315,NaN,2.0 Liter TFSI,Automatic,Daytona Gray,Black,1,0,17498
8908,8908,BMW,M3 CS,2022,13777,NaN,453.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Automatic,Gray,Black,0,1,91000
9522,9522,Kia,EV6 Wind,2022,4514,NaN,1.6L I4 16V GDI DOHC Turbo,Automatic,Pacific Blue Metallic,Satin Black,0,0,47598


Interestingly there is also another value for missing value in this column: `-`. Lets also invastigate this one. There are only few of these rows where the enigne is not missing as well. Therefore we will drop the rows where both fuel and engine are missing. The others we will fill with `Gasoline`.

In [53]:
print(data_others[data_others['fuel_type'] == '–'].shape)
wrong_ids = data_others[data_others['fuel_type'] == '–'][data_others['engine'] == '–']['id']

data_others.drop(wrong_ids, inplace = True)

(781, 13)


/tmp/ipykernel_11707/1827019610.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  wrong_ids = data_others[data_others['fuel_type'] == '–'][data_others['engine'] == '–']['id']


Here we can see that these cars seem to be almost exclusivly gasoline. We will just overwrite the values as `Gasoline` there might be some wrong classified examples in the pool, but this is just such a low percentage of our dataset, that we will just ignore this possible mistake.

In [54]:
for ind in data_others['id']:
    if type(data_others.loc[ind, 'fuel_type']) != str or type(data_others.loc[ind, 'fuel_type']) == '–':
        if 'Electric' in data_others.loc[ind, 'engine'] or data_others.loc[ind, 'brand'] == 'Tesla':
            data_others.loc[ind, 'fuel_type'] = 'Electric'
        else:
            data_others.loc[ind, 'fuel_type'] = 'Gasoline'

Now we will have to use one hot encoding for this.

In [55]:
fuel_types = set()
for fuel in data_others['fuel_type']:
    fuel_types.add(fuel.lower())

for fuel in fuel_types:
    data_color['fuel_' + fuel] = 0
    
for ind, fuel in enumerate(data_others['fuel_type']):
    data_others.loc[ind, 'fuel_' + fuel] = 1


In [56]:
data_others.drop(['id', 'fuel_type', 'brand', 'model', 'engine', 'transmission', 'ext_col', 'int_col'], axis=1, inplace=True)

## Transmission

Now we will look at the transmission. Here we can firstly extract the number of gears given.

In [57]:
data_transmission = data[['id','transmission']].copy()

data_transmission['gears'] = np.nan
data_transmission['Automatic'] = 0
data_transmission['Manual'] = 0

ignore_information = ['Transmission', # These are either redundant information or don't contain any information
                      'Shift',
                      'Mode',
                      'SCEDHULED',
                      'FOR',
                      'OR',
                      'IN',
                      'with',
                      'O',
                      'Controlled',
                      'Fixed',
                      'Gear',
                      'Auto-Shift',
                      '–',
                      '6',
                      'SCHEDULED'
                     ]

In [58]:
automatic_names = ['A/T', 'AT', 'Automatic']
manual_names = ['Mt', 'M/T', 'Manual']

for ind, transmission in enumerate(data_transmission['transmission']):
    infos = transmission.split(' ')
    for info in infos:
        if info not in ignore_information:
            if '-Speed' in info:
                number = ''
                orig_info = info
                while info[0].isnumeric() and len(info) > 0:
                    number += info[0]
                    info = info[1:]
                if len(number) > 0:
                    data_transmission.loc[ind, 'gears'] = np.int32(number)
                elif orig_info[:6] == 'Single':
                    data_transmission.loc[ind, 'gears'] = np.int32(1)
            elif info == 'Speed':
                data_transmission.loc[ind, 'gears'] = np.int32(6) # Manually checked that this is always the case 
            elif info == '2':
                data_transmission.loc[ind, 'gears'] = np.int32(2) # Assume that this is the number of gears
            elif info == '8-SPEED':
                data_transmission.loc[ind, 'gears'] = np.int32(8)
            elif info in automatic_names:
                data_transmission.loc[ind, 'Automatic'] = 1
            elif info in manual_names:
                data_transmission.loc[ind, 'Manual'] = 1
            else:
                data_transmission.loc[ind, info] = 1
        
data_transmission.head()

,id,transmission,gears,Automatic,Manual,w/Dual,CVT,Overdrive,Switch,Electronically,F,CVT-F,At/Mt,Variable,DCT,PRODUCTION
0,0,A/T,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,A/T,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,A/T,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Transmission w/Dual Shift Mode,NaN,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,7-Speed A/T,7.0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we will again have to fill up the missing values. For the gears we will use the mode, for the others we will just replace them with 0, as they are one hot encoding.

In [59]:
mode = data_transmission['gears'].mode()[0]

print(mode)

data_transmission['gears'] = [val if not pd.isna(val) else mode for val in data_transmission['gears']]

data_transmission.fillna(0, inplace=True)

data_transmission.drop(['id', 'transmission'], axis=1, inplace=True)
data_transmission.head()

6.0


,gears,Automatic,Manual,w/Dual,CVT,Overdrive,Switch,Electronically,F,CVT-F,At/Mt,Variable,DCT,PRODUCTION
0,6.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Color

Lets now take a look at the colors. Here we have to bear in mind that there are two features with color `int_col` und `ext_col`. Lets now look at the different values for these features. We will start with the `int_col`

In [60]:
data_color = data[['int_col', 'ext_col']].copy()

In [61]:
data_color.head()

,int_col,ext_col
0,Gray,Yellow
1,Beige,Silver
2,Gray,Blue
3,Black,Black
4,Beige,Black


In [62]:
int_colors = set()

for color in data_color['int_col']:
    int_colors.add(color)

int_colors.remove('–')
print(int_colors)

{'Gideon', 'Gold', 'Sakhir Orange/Black', 'Ebony.', 'Caramel', 'Amber', 'Tan/Ebony', 'Shara Beige', 'Tan/Ebony/Ebony', 'Ebony / Pimento', 'Hotspur Hide', 'Very Light Cashmere', 'BLACK', 'Parchment.', 'Camel', 'Deep Garnet', 'Charcoal', 'Porpoise', 'Rioja Red', 'Rock Gray', 'Orchid', 'Black / Stone Grey', 'White / Brown', 'Cobalt Blue', 'Black / Graphite', 'White', 'Black / Express Red', 'Saddle Brown', 'Bianco Polar', 'Black', 'Nero Ade', 'Ebony/Light Oyster Stitch', 'Silk Beige/Black', 'Graphite', 'Light Slate', 'Espresso', 'Silk Beige/Espresso Brown', 'Sand Beige', 'Black w/Red Stitching', 'Macchiato', 'Chateau', 'Grace White', 'Satin Black', 'Parchment', 'Medium Earth Gray', 'Medium Pewter', 'Rhapsody Blue', 'Sport', 'Red', 'Portland', 'Carbon Black', 'Medium Dark Slate', 'Saiga Beige', 'BEIGE', 'Charcoal Black', 'Dark Ash', 'Deep Chestnut', 'Red / Black', 'Ceramic', 'Roast', 'Sakhir Orange', 'Red/Black', 'Beluga Hide', 'Orange', 'Tupelo', 'Graphite w/Gun Metal', 'Ebony / Ebony Acce

We will firstly convert all the colors to lower case letters and then remove any dots that are in the names. We will fill up '–'-values with random values from that column. Then we will split the strings at 'w/', '/' and ' ' and use one hot encoding for the remaining strings.

In [63]:
import random

int_colors_split = set()
for ind, color in enumerate(data_color['int_col']):
    while data_color.loc[ind, 'int_col'] == '–':
        data_color.loc[ind, 'int_col'] = random.choice(data_color['int_col'])
    color_string = data_color.loc[ind, 'int_col'].lower().replace('.','')
        
    values = color_string.split('w/')
    new_values = []
    for value in values:
        new_values += value.split('/')
    newer_values = []
    for value in new_values:
        newer_values += value.split(' ')
    for value in newer_values:
        int_colors_split.add(value.replace(' ', ''))

int_colors_split.remove('')
print(int_colors_split)
data_color.head()

{'mistral', 'stitch', 'jet', 'ceramic', 'charles', 'charcoal', 'contrast', 'cocoa', 'taurus', 'agave', 'tupelo', 'kyalami', 'ash', 'caramel', 'almond', 'gold', 'linen', 'cashmere', 'stitching', 'silver', 'orange', 'ebony', 'earth', 'grace', 'porpoise', 'global', 'aragon', 'bolsters', 'shale', 'titanium', 'auburn', 'adrenaline', 'sandstone', 'dark', 'parchment', 'light', 'giallo', 'brandy', 'medium', 'boulder', 'carbon', 'silk', 'platinum', 'beluga', 'rioja', 'rhapsody', 'pimento', 'leather', 'sand', 'sahara', 'black', 'sakhir', 'very', 'gun', 'mocha', 'galvanized', 'amber', 'shara', 'classic', 'red', 'cappuccino', 'tension', 'yellow', 'sarder', 'blue', 'polar', 'white', 'garnet', 'camel', 'pearl', 'tan', 'walnut', 'ice', 'macchiato', 'satin', 'green', 'hide', 'sport', 'deep', 'metal', 'orchid', 'mountain', 'hotspur', 'amg', 'chestnut', 'beige', 'anthracite', 'graphite', 'graystone', 'bianco', 'espresso', 'accents', 'pier', 'canberra', 'cobalt', 'chateau', 'grey', 'nero', 'cypress', 'ex

,int_col,ext_col
0,Gray,Yellow
1,Beige,Silver
2,Gray,Blue
3,Black,Black
4,Beige,Black


We will now use one hot encoding for this.

In [64]:
for col in int_colors_split:
    data_color['int_' + col] = 0
    
for ind, color in enumerate(data_color['int_col']):
    values = data_color.loc[ind, 'int_col'].lower().replace('.','').split('w/')
    new_values = []
    for value in values:
        new_values += value.split('/')
    newer_values = []
    for value in new_values:
        newer_values += value.split(' ')
    for value in newer_values:
        if value != '':
            data_color.loc[ind, 'int_' + value] = 1

data_color.head()

/tmp/ipykernel_11707/2754218350.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_color['int_' + col] = 0
/tmp/ipykernel_11707/2754218350.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_color['int_' + col] = 0
/tmp/ipykernel_11707/2754218350.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.c

,int_col,ext_col,int_mistral,int_stitch,int_jet,int_ceramic,int_charles,int_charcoal,int_contrast,int_cocoa,...,int_titan,int_onyx,int_blk,int_roast,int_gray,int_portland,int_mesa,int_ade,int_raven,int_rock
0,Gray,Yellow,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,Beige,Silver,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Gray,Blue,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,Black,Black,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Beige,Black,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Lets now do the same thing for the exterior color.

In [65]:
ext_colors_split = set()
for ind, color in enumerate(data_color['ext_col']):
    while data_color.loc[ind, 'ext_col'] == '–':
        data_color.loc[ind, 'ext_col'] = random.choice(data_color['ext_col'])
    color_string = data_color.loc[ind, 'ext_col'].lower().replace('.','')
        
    values = color_string.split('w/')
    new_values = []
    for value in values:
        new_values += value.split('/')
    newer_values = []
    for value in new_values:
        newer_values += value.split(' ')
    for value in newer_values:
        ext_colors_split.add(value.replace(' ', ''))

ext_colors_split.remove('')

for col in ext_colors_split:
    data_color['ext_' + col] = 0
    
for ind, color in enumerate(data_color['ext_col']):
    values = data_color.loc[ind, 'ext_col'].lower().replace('.','').split('w/')
    new_values = []
    for value in values:
        new_values += value.split('/')
    newer_values = []
    for value in new_values:
        newer_values += value.split(' ')
    for value in newer_values:
        if value != '':
            data_color.loc[ind, 'ext_' + value] = 1



/tmp/ipykernel_11707/3318554533.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_color['ext_' + col] = 0
/tmp/ipykernel_11707/3318554533.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_color['ext_' + col] = 0
/tmp/ipykernel_11707/3318554533.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = fram

In [66]:
data_color.drop(['int_col', 'ext_col'], axis=1, inplace=True)
data_color.head()

,int_mistral,int_stitch,int_jet,int_ceramic,int_charles,int_charcoal,int_contrast,int_cocoa,int_taurus,int_agave,...,ext_radiant,ext_hyper,ext_ruby,ext_caelum,ext_raven,ext_vik,ext_rock,ext_moss,ext_tri-coat,ext_oryx
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Brand and Model

We will use one-hot encoding for the brand. For our first tries we will drop the model since there are too many different possible values for this.

In [67]:
data_brand = pd.DataFrame(data['brand'].copy())
brands = set()

for ind, brand in enumerate(data_brand['brand']):
    brands.add(brand.lower())

print(brands)
for brand in brands:
    data_brand[brand] = 0

for ind, brand in enumerate(data_brand['brand']):
    data_brand.loc[ind, brand.lower()] = 1

data_brand.drop('brand', axis=1, inplace=True)
data_brand.head()

{'lotus', 'scion', 'tesla', 'lucid', 'porsche', 'subaru', 'volvo', 'suzuki', 'lamborghini', 'genesis', 'lincoln', 'aston', 'toyota', 'volkswagen', 'chrysler', 'chevrolet', 'mitsubishi', 'maybach', 'pontiac', 'audi', 'dodge', 'hummer', 'honda', 'smart', 'land', 'plymouth', 'mercury', 'alfa', 'ford', 'lexus', 'buick', 'polestar', 'hyundai', 'saab', 'infiniti', 'karma', 'mercedes-benz', 'jaguar', 'ram', 'mini', 'cadillac', 'rolls-royce', 'nissan', 'mazda', 'ferrari', 'acura', 'bentley', 'mclaren', 'fiat', 'bugatti', 'saturn', 'rivian', 'bmw', 'jeep', 'gmc', 'maserati', 'kia'}


,lotus,scion,tesla,lucid,porsche,subaru,volvo,suzuki,lamborghini,genesis,...,mclaren,fiat,bugatti,saturn,rivian,bmw,jeep,gmc,maserati,kia
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Put it together

Now as a final step we will concatenate all the dataframes we have created up to this point and save it as our data.

In [68]:
data_final = pd.concat([data_engine, data_others, data_transmission, data_color, data_brand], axis=1)


In [69]:
data_final.head()

,horsepower,liters,cylinders,valves,V,I,H,W,L/406,L/122,...,mclaren,fiat,bugatti,saturn,rivian,bmw,jeep,gmc,maserati,kia
0,172.0,1.6,4.0,16.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,252.0,3.9,8.0,32.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,320.0,5.3,8.0,32.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,420.0,5.0,8.0,32.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,208.0,2.0,4.0,16.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
data_final.to_csv('data_final.csv')